In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Load the data
df = pd.read_csv('train.csv')

# Extract the relevant columns
text_columns = [
    'Girisimcilikle Ilgili Deneyiminiz Var Mi?',
    'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'
]
target_column = 'Degerlendirme Puani'

# Drop rows with missing target values
df = df.dropna(subset=[target_column])

# Fill missing values in text columns with empty strings
df[text_columns] = df[text_columns].fillna('')

# Combine text columns into one for processing
df['combined_text'] = df[text_columns].apply(lambda x: ' '.join(x), axis=1)

# Tokenize and preprocess the text data
def preprocess_text(text):
    return simple_preprocess(text, deacc=True)

# Prepare the Word2Vec model
sentences = [preprocess_text(text) for text in df['combined_text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Create document vectors by averaging word vectors
def vectorize_text(text):
    tokens = preprocess_text(text)
    vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
    if len(vectors) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(vectors, axis=0)

df['text_vector'] = df['combined_text'].apply(vectorize_text)
X = np.array(df['text_vector'].tolist())
y = df[target_column]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

# Print actual vs predicted values
print("\nSample Actual vs. Predicted Values:")
for actual, predicted in zip(y_test.head(10), y_pred[:10]):
    print(f"Actual: {actual}, Predicted: {predicted}")

# Optionally, save the model for future use
import joblib
joblib.dump(model, 'regression_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


/var/folders/0p/hymvzzjx3cb75ylj_41b1d8h0000gn/T/ipykernel_73641/643811995.py:11: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


Mean Squared Error: 283.37

Sample Actual vs. Predicted Values:
Actual: 36.0, Predicted: 41.532094513297814
Actual: 68.0, Predicted: 42.848110594715045
Actual: 7.0, Predicted: 28.271996836870194
Actual: 30.0, Predicted: 44.11563740836077
Actual: 11.0, Predicted: 28.271996836870194
Actual: 46.0, Predicted: 27.320539015448325
Actual: 31.0, Predicted: 28.271996836870194
Actual: 21.0, Predicted: 43.87453393452943
Actual: 15.0, Predicted: 28.271996836870194
Actual: 15.0, Predicted: 27.320539015448325


['scaler.pkl']